In [1]:
"""
Fetures of this chatbot:
    1.Will able to have conversation
    2.will remember previous interaction
    3.will have diffrent memeory for diffrent session
    4.will able to remember limited number of messages
"""

'\nFetures of this chatbot:\n    1.Will able to have conversation\n    2.will remember previous interaction\n    3.will have diffrent memeory for diffrent session\n    4.will able to remember limited number of messages\n'

In [ ]:
import os
from dotenv import load_dotenv,find_dotenv
_=load_dotenv(find_dotenv())
google_api_key=os.environ['GEMINI_API_KEY']

from langchain_google_genai import ChatGoogleGenerativeAI
chatbot = ChatGoogleGenerativeAI(model="gemini-1.5-pro",api_key=google_api_key)

#testing
response=chatbot.invoke("Write capital of Bihar")
print(response.content)

In [ ]:
from langchain_core.messages import HumanMessage
messagetobot=[
    HumanMessage(content="My favourite food is samosa")
]

chatbot.invoke(messagetobot)

AIMessage(content="That's great! Samosas are a delicious and popular snack/appetizer.  What kind of samosas do you like best? (e.g., potato and pea, meat, etc.)", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-d931d6ec-bca2-4d40-a85a-97e6d0463a9f-0', usage_metadata={'input_tokens': 7, 'output_tokens': 43, 'total_tokens': 50, 'input_token_details': {'cache_read': 0}})

In [22]:
messagetobot1=[
    HumanMessage(content="What is my favourite food")
]
chatbot.invoke(messagetobot1)


AIMessage(content="I have no idea what your favorite food is. I don't have access to your personal information or preferences.  If you tell me what your favorite food is, I can remember it for our current conversation, but I won't retain that information after our conversation ends.", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-8b72b417-e74b-417d-9fdd-24d632f8e0b6-0', usage_metadata={'input_tokens': 6, 'output_tokens': 57, 'total_tokens': 63, 'input_token_details': {'cache_read': 0}})

In [23]:
"""
Here we will use:
we will use ChatMessageHistory Package
we will save chatbot memory in a python dictionary called chatbotMemory
we will define get_session_history function to create a session id for each conversation
we will use built -in runnable With Mesage History
"""

'\nHere we will use:\nwe will use ChatMessageHistory Package\nwe will save chatbot memory in a python dictionary called chatbotMemory\nwe will define get_session_history function to create a session id for each conversation\nwe will use built -in runnable With Mesage History\n'

In [24]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory


chatbotmemory={}

def get_session_history(session_id:str) -> BaseChatMessageHistory:
    if session_id not in chatbotmemory:
        chatbotmemory[session_id]=ChatMessageHistory()
    return chatbotmemory[session_id]

chat_with_mesage_history=RunnableWithMessageHistory(
    chatbot,
    get_session_history
)



In [25]:
session1={"configurable":{"session_id":"001"}}

In [ ]:
responsefromchatbot=chat_with_mesage_history.invoke(
    [HumanMessage(content="My favourite color is red")],
    config=session1
)
responsefromchatbot.content

Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..


"That's great! Red is a vibrant and popular color.  It's often associated with energy, passion, and excitement."

In [28]:
responsefromchatbot1=chat_with_mesage_history.invoke(
    [HumanMessage(content="What is my favouriteb color")],
    config=session1
)
responsefromchatbot1.content

Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..


ResourceExhausted: 429 Resource has been exhausted (e.g. check quota).

In [12]:
#Make another session and test it
session2={"configurable":{"session_id":"002"}}
responsefromchatbot1=chat_with_mesage_history.invoke(
    [HumanMessage(content="What is my favouriteb color")],
    config=session2
)
responsefromchatbot1.content


"I have no way of knowing your favorite color.  I'm a large language model, and I don't have access to your personal information or preferences."

In [13]:
#Limiting number of remebered vconversation


from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

from langchain_core.runnables import RunnablePassthrough


def limited_memory_of_messages(messages, number_of_messages_to_keep=2):
    return messages[-number_of_messages_to_keep:]

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

limitedMemoryChain = (
    RunnablePassthrough.assign(messages=lambda x: limited_memory_of_messages(x["messages"]))
    | prompt 
    | chatbot
)



In [14]:
chatbot_with_limited_message_history = RunnableWithMessageHistory(
    limitedMemoryChain,
    get_session_history,
    input_messages_key="messages",
)

In [ ]:
responseFromChatbot1= chatbot_with_limited_message_history.invoke(
    {
        "messages": [HumanMessage(content="what My favorite city?")],
    },
    config=session1,
)

responseFromChatbot1.content

Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised RetryError: Timeout of 600.0s exceeded, last exception: 503 failed to connect to all addresses; last error: UNKNOWN: ipv4:142.250.196.10:443: Ssl handshake failed (TSI_PROTOCOL_FAILURE): SSL_ERROR_SSL: error:1000007d:SSL routines:OPENSSL_internal:CERTIFICATE_VERIFY_FAILED: self signed certificate in certificate chain.


RetryError: Timeout of 600.0s exceeded, last exception: 503 failed to connect to all addresses; last error: UNKNOWN: ipv4:142.250.193.138:443: socket is null